In [1]:
import pandas as pd
import datetime

In [2]:
pd.options.display.max_columns=500
pd.options.display.max_colwidth = None
pd.options.display.max_rows = None

In [3]:
result = []
render = False

In [4]:
from network import DeepmindNet
from torchsummary import summary

summary(DeepmindNet(), (3,84,84))

C:\Users\sinny\anaconda3\envs\rl-learn\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 20, 20]           6,176
       BatchNorm2d-2           [-1, 32, 20, 20]              64
              ReLU-3           [-1, 32, 20, 20]               0
            Conv2d-4             [-1, 64, 9, 9]          32,832
       BatchNorm2d-5             [-1, 64, 9, 9]             128
              ReLU-6             [-1, 64, 9, 9]               0
            Conv2d-7             [-1, 64, 7, 7]          36,928
       BatchNorm2d-8             [-1, 64, 7, 7]             128
              ReLU-9             [-1, 64, 7, 7]               0
           Linear-10                  [-1, 512]       1,606,144
             ReLU-11                  [-1, 512]               0
          Dropout-12                  [-1, 512]               0
           Linear-13                    [-1, 9]           4,617
             ReLU-14                   

In [5]:
def print_result(df, num=10):
    
    df = df.copy()
    
    for i in range(num, len(df), num):
        print(f"episode {i} average reward: {df[i-num:i].mean()['reward']} max reward: {df[i-num:i].max()['reward']}")

### Global Varaiable & Property

In [6]:
# global variable


agent = None
target_epsilon = None
savefile = None 

# global property

render = False
show_action = False

### CODE

In [7]:
import time
import pandas as pd
from dqn import *

SAVE_MODEL_DIR = "./model/20220529"
SAVE_MODEL_PREFIX = "Pacman-ai-"

parameter = HyperParameter(
    batch_size=32,  # 한 학습당 사용되는 데이터 개수 (batch 크기)
    buffer_limit=100000,  # replay buffer 최대 사이즈
    gamma=0.98,  # 감쇄율 (Q-learning 포함)
    learning_rate=0.00025,  # 학습률
    epoch=100,
    update_period=100,
)

# global variable

agent = None
savefile = None
render = False

csv_interval = 100
save_interval = 1000

result = []

if __name__ == '__main__':

    env = Environment()
    agent = DQNAgent(param=parameter)

    n_epi = 0

    while True:

        # e-greedy 행동 결정에 사용되는 epsilon, 에피소드가 지날 수록 값을 줄어나가게 함
        epsilon = max(0.1, 1 - n_epi / 100000)

        state = env.reset()
        done = False

        action_list = []
        episode_reward = 0
        loss = -1

        while not done:
            # agent가 결정한 다음 행동
            action = agent.predict(state, epsilon)

            # 행동을 취한 후 결과로 S', R, 메타정보 등을 받음
            state_prime, reward, done, info = env.step(action)

            agent.memorize(state, state_prime, action, reward)

            state = state_prime

            episode_reward += reward

            if render:
                env.render()
                time.sleep(0.01)

        # 에피소드가 끝날때마다 학습을 진행
        # 초기에는 학습 데이터가 적고, 학습 가치가 적으므로 충분히 Replay Buffer 에 쌓은 다음 학습 수행
        if n_epi > 3:
            loss = agent.train().mean()

        if n_epi % csv_interval == 0 and n_epi != 0:
            pd.DataFrame(result).to_csv(f'{SAVE_MODEL_DIR}/result.csv')

        if n_epi % save_interval == 0 and n_epi != 0:
            agent.save(f"{SAVE_MODEL_DIR}/{SAVE_MODEL_PREFIX}{n_epi}.pi")
            score = 0.0

        result.append({
            'loss': loss,
            'reward': episode_reward,
            'epsilon': epsilon
        })

        n_epi += 1


In [30]:
df = pd.DataFrame(result)
df

,reward,epsilon
0,30,1.00000
1,70,0.99999
2,90,0.99998
3,10,0.99997
4,50,0.99996
5,40,0.99995
6,50,0.99994
7,160,0.99993
8,100,0.99992
9,110,0.99991


In [9]:
print_result(df, num=100)

In [31]:
render = True

In [32]:
render = False

n_episode :16400, score : 79.4, eps : 83.6%
n_episode :16500, score : 72.6, eps : 83.5%
n_episode :16600, score : 82.9, eps : 83.4%
n_episode :16700, score : 77.5, eps : 83.3%
n_episode :16800, score : 60.2, eps : 83.2%
n_episode :16900, score : 85.2, eps : 83.1%
n_episode :17000, score : 64.9, eps : 83.0%
n_episode :17100, score : 88.4, eps : 82.9%
n_episode :17200, score : 83.8, eps : 82.8%
n_episode :17300, score : 73.1, eps : 82.7%
n_episode :17400, score : 76.3, eps : 82.6%
n_episode :17500, score : 90.9, eps : 82.5%
n_episode :17600, score : 90.7, eps : 82.4%
n_episode :17700, score : 81.4, eps : 82.3%
n_episode :17800, score : 83.1, eps : 82.2%
n_episode :17900, score : 83.7, eps : 82.1%
n_episode :18000, score : 91.7, eps : 82.0%
n_episode :18100, score : 84.9, eps : 81.9%
n_episode :18200, score : 89.2, eps : 81.8%


In [12]:
raise Exception

C:\Users\sinny\anaconda3\envs\rl-learn\lib\site-packages\gym\envs\atari\environment.py:269: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  "We strongly suggest supplying `render_mode` when "


Exception: 

In [ ]:
agent.save('20220527082400_77472epi.pt')

In [ ]:
# 45176까지 하였었음

[20220517 084300]
유령 먹는 점수가 eat 점수에 비해 커가지고 agent가 야비하게 아이템을 먹고 쳐 기다린다... 보상점수를 좀 줄여야 할 듯 하다.
결국엔 전체적인 게임 흐름이 아니라 독립되어 한 상태에서 보상 값읅 구하는거기때문에 유령 먹는점수를 하향하는게 매우 중요하다
왜 계속 구석에 가있나 햇네 ㅋㅋㅋ


ghost_reward 변수를 새롭게 정의후 200에서 0으로 수정 유령은 필수로 먹어야하는 요소가 아니며 살아남기 위한 요소라거 정의하도록 한다.

[20220518 083000] 한 목숨 당 death 제어하는거 제외 이유는 이전에 높은 점수를 얻었는데도 불구하고
해당 구조로 인해 쿠키를 많이 먹지 못한 시나리오에 비해 우선도가 떨어질수도 있기 때문\

빨리 죽을 수 있는 문제는 엡실론 값을 주어 죽는게 아닌 다른 방법을 취해 쿠키를 먹었을때 보상을 얻어 점수가 쌓이게 되닌깐 결국엔 그방향ㅇ
으로 갈것이라 예상됨

[20220518 121600] 왜 계속 팩맨이 구석탱이로 가나 생각해보니 데스에 대한 마이너스 값을 너무 크게 해서 팩맨이 소녀감성을 갖게되서
발생된 일이 아닌가 싶음... 타켓은 다음 상태를 의미하기 때문에 전체적인 관점에선 아닐 수 있으나 단일 시점에선 그럴 가능성이 있음 사망 값을 좀 낮추어볼까...? (Q-learning을 사용한다면 전체적인 시점이 아닌 짧은 시점에서 찾아보는 것은 어떨까 애초에 td(시간차 방식)으로 측정하므로

함수 근사치로 구한다는 것을 명심하자

[20220520 143200] 엡실론 값을 너무 적게 해서 다양한 방법을 찾지못해 한 쪽으로 솔리는 것은 아닐까..?

확실시 .SCROE 값은 줄어들음

EPSILON = 0.5 START
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
================================================================
            Conv2d-1           [-1, 16, 83, 83]             208
       BatchNorm2d-2           [-1, 16, 83, 83]              32
              ReLU-3           [-1, 16, 83, 83]               0
         MaxPool2d-4           [-1, 16, 41, 41]               0
            Conv2d-5           [-1, 32, 40, 40]           2,080
       BatchNorm2d-6           [-1, 32, 40, 40]              64
              ReLU-7           [-1, 32, 40, 40]               0
         MaxPool2d-8           [-1, 32, 13, 13]               0
            Linear-9                 [-1, 1024]       5,538,816
             ReLU-10                 [-1, 1024]               0
           Linear-11                  [-1, 256]         262,400
             ReLU-12                  [-1, 256]               0
           Linear-13                    [-1, 9]           2,313


[20220522] 처음에 epsilon 값을 0.5 값으로 높게 준 다음 좀 어느정도 시나리오가 지나면 0.2로..?

[20220522125500] q_action 값과 target 값의 차이를 구하는 것인데 
death 값을 너무 크게 주지 최대한 쉽ㄱ ㅔ맞출려고 일부러 죽는 모습이 보임... 보상보다 큰 abs() 값 차이를 두어선 안될듯
death 값을 너무 크게 두니 abs() 값이 크다보니깐 - >왜냐면 loss 값 계산시 abs() 차이를 보기 때문...
오히려 계속 괴물한테 처음에 한번 죽고나서 계속 죽을려고함...

처음에 접근하면 50일테니깐 보상 값을 크게 하고 죽음이나 아무 의미 없는 값은 적게 가져가는것이 좋을듯

[20220523175800]
reward = 10, death = -5, move = -1
20220523175600.pt 

[20220524081600] 중간점검 reward 100 death -50 move= -1

[20220525082000] 현재 그 상황에서 생각하자 애총 최대 값을 가져가기 때문에 20220522125500 은 틀린 말임 보상과 죽음이 있다면 보상보다 죽음을 피한
는것이 옳은 방법 일부러 죽으러 간다는 것은 말이 안됨 따라서 reward = 10, death = -50 move = 0으로 한다.
move는 갔던 길을 다시 돌아가야 할 상황이 올 수 있으므로 아래로 갔다 위로 가야 하는 경우 1000점 이상이 나와야 높게 가져가 수준 만약에
해당 방법이 성공한다면 교훈은 너무 빡세게 가져가지 말자 느슨하게 가자

[20220525172400] 중간 세이브 20220525172400.pt

[20220527150000] 보상 갑 수정은 구조 자체를 변경하지 않는 한 가망이 없을 것으로 보임